In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor

from statsmodels.stats.outliers_influence import variance_inflation_factor




%matplotlib inline

/Users/clairesarraille/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Read in Data:
df = pd.read_csv('data/kc_house_data.csv')

### King County, Washington Residential Glossary of Terms:
- https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#s


### The King County Grading System for Buildings:
#### Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:

1-3 Falls short of minimum building standards. Normally cabin or inferior structure.

4 Generally older, low quality construction. Does not meet code.

5 Low construction costs and workmanship. Small, simple design.

6 Lowest grade currently meeting building code. Low quality materials and simple designs.

7 Average grade of construction and design. Commonly seen in plats and older sub-divisions.

8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

9 Better architectural design with extra interior and exterior design and quality.

10 Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

13 Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

### Business Case:
- Goal is to be able to suggest house improvements to average-income home-owners to improve their selling price
- Potentially targeting:
    - Bought home 20+ years ago, kids off to college, now want to sell home & down-size
    - If your home is older, how do you sell it for as much as possible?
    - What types of renovations are worth the investment?
        - Would the cost of adding a bathroom be worth the increase in selling price?

In [4]:
# There are 176 ids that repeat in this dataset:
repeating_ids = pd.DataFrame(df.id.value_counts()[df.id.value_counts() > 1])

In [5]:
df.sort_values('id', ascending = True).head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2495,1000102,4/22/2015,300000.0,6,3.00,2400,9373,2.0,0.0,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
2494,1000102,9/16/2014,280000.0,6,3.00,2400,9373,2.0,NaN,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
6729,1200019,5/8/2014,647500.0,4,1.75,2060,26036,1.0,NaN,0.0,...,8,1160,900.0,1947,0.0,98166,47.4444,-122.351,2590,21891
8404,1200021,8/11/2014,400000.0,3,1.00,1460,43000,1.0,0.0,0.0,...,7,1460,0.0,1952,0.0,98166,47.4434,-122.347,2250,20023
8800,2800031,4/1/2015,235000.0,3,1.00,1430,7599,1.5,0.0,0.0,...,6,1010,420.0,1930,0.0,98168,47.4783,-122.265,1290,10320
3553,3600057,3/19/2015,402500.0,4,2.00,1650,3504,1.0,0.0,0.0,...,7,760,890.0,1951,2013.0,98144,47.5803,-122.294,1480,3504
18506,3600072,3/30/2015,680000.0,4,2.75,2220,5310,1.0,NaN,0.0,...,7,1170,1050.0,1951,NaN,98144,47.5801,-122.294,1540,4200
3197,3800008,2/24/2015,178000.0,5,1.50,1990,18200,1.0,NaN,0.0,...,7,1990,0.0,1960,NaN,98178,47.4938,-122.262,1860,8658
21047,5200087,7/9/2014,487000.0,4,2.50,2540,5001,2.0,0.0,0.0,...,9,2540,0.0,2005,0.0,98108,47.5423,-122.302,2360,6834
4329,6200017,11/12/2014,281000.0,3,1.00,1340,21336,1.5,0.0,0.0,...,5,1340,0.0,1945,0.0,98032,47.4023,-122.273,1340,37703


In [6]:
df.id.value_counts()[df.id.value_counts() > 1]

795000620     3
1825069031    2
2019200220    2
7129304540    2
1781500435    2
             ..
7893805650    2
8161020060    2
1432400120    2
7701960990    2
1788900230    2
Name: id, Length: 176, dtype: int64

In [7]:
id_dupe_bools = df['id'].duplicated(keep = False)
df_dupe_id = df[id_dupe_bools] 


In [58]:
df_dupe_id

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
93,6021501535,7/25/2014,430000.0,3,1.50,1580,5000,1.0,0.0,0.0,...,8,1290,290.0,1939,0.0,98117,47.6870,-122.386,1570,4500
94,6021501535,12/23/2014,700000.0,3,1.50,1580,5000,1.0,0.0,0.0,...,8,1290,290.0,1939,0.0,98117,47.6870,-122.386,1570,4500
313,4139480200,6/18/2014,1380000.0,4,3.25,4290,12103,1.0,0.0,3.0,...,11,2690,1600.0,1997,0.0,98006,47.5503,-122.102,3860,11244
314,4139480200,12/9/2014,1400000.0,4,3.25,4290,12103,1.0,0.0,3.0,...,11,2690,1600.0,1997,0.0,98006,47.5503,-122.102,3860,11244
324,7520000520,9/5/2014,232000.0,2,1.00,1240,12092,1.0,NaN,0.0,...,6,960,280.0,1922,1984.0,98146,47.4957,-122.352,1820,7460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20654,8564860270,3/30/2015,502000.0,4,2.50,2680,5539,2.0,NaN,0.0,...,8,2680,0.0,2013,0.0,98045,47.4759,-121.734,2680,5992
20763,6300000226,6/26/2014,240000.0,4,1.00,1200,2171,1.5,0.0,0.0,...,7,1200,0.0,1933,0.0,98133,47.7076,-122.342,1130,1598
20764,6300000226,5/4/2015,380000.0,4,1.00,1200,2171,1.5,0.0,0.0,...,7,1200,0.0,1933,0.0,98133,47.7076,-122.342,1130,1598
21564,7853420110,10/3/2014,594866.0,3,3.00,2780,6000,2.0,0.0,0.0,...,9,2780,0.0,2013,0.0,98065,47.5184,-121.886,2850,6000


In [8]:
df_dupe_id[['id', 'date', 'condition', 'grade', 'bathrooms', 'bedrooms', 'sqft_living']]

,id,date,condition,grade,bathrooms,bedrooms,sqft_living
93,6021501535,7/25/2014,3,8,1.50,3,1580
94,6021501535,12/23/2014,3,8,1.50,3,1580
313,4139480200,6/18/2014,3,11,3.25,4,4290
314,4139480200,12/9/2014,3,11,3.25,4,4290
324,7520000520,9/5/2014,3,6,1.00,2,1240
...,...,...,...,...,...,...,...
20654,8564860270,3/30/2015,3,8,2.50,4,2680
20763,6300000226,6/26/2014,3,7,1.00,4,1200
20764,6300000226,5/4/2015,3,7,1.00,4,1200
21564,7853420110,10/3/2014,3,9,3.00,3,2780
